## CNN
Referencing data downloaded from https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia/version/2

### Installing the requirements
```python
pip install keras
pip install tensorflow
```

### Import relevant modules

In [1]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

Using TensorFlow backend.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


### Config

In [2]:
dimension = 128
batch_size = 16

### Setup

In [3]:
# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (dimension, dimension, 3), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

# Compiling the CNN
classifier.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Fitting the model using train images

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(
    '../image_data/chest_xray/train/',
    target_size = (dimension, dimension),
    batch_size = batch_size,
    class_mode = 'binary'
)

test_set = test_datagen.flow_from_directory(
    '../image_data/chest_xray/test/',
    target_size = (dimension, dimension),
    batch_size = batch_size,
    class_mode = 'binary'
)

classifier.fit_generator(
    training_set,
    steps_per_epoch = 1200,
    epochs = 5,
    validation_steps = 800,
    validation_data = test_set,
)

Found 5102 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Epoch 1/5
 626/1200 [==============>...............] - ETA: 4:42 - loss: 4.0369 - acc: 0.7463

### Making Predictions

In [ ]:
import numpy
from keras.preprocessing import image

In [ ]:
training_set.class_indices

In [ ]:
import os
predict_roots = ['../image_data/chest_xray/val/NORMAL/','../image_data/chest_xray/val/PNEUMONIA/']
results = []
for path in predict_roots:
    
    correct_class = path.split('/')[4]
    
    for filename in os.listdir(path):
        if '.png' in filename or '.jpeg' in filename:
            print('Processing {}'.format(filename))
            test_image = image.load_img(
                path + filename, 
                target_size = (dimension, dimension)
            )

            test_image = image.img_to_array(test_image)
            test_image = numpy.expand_dims(test_image, axis = 0)
            result = classifier.predict(test_image)

            if result[0][0] == 1:
                prediction = 'PNEUMONIA'
            else:
                prediction = 'NORMAL'
                
            if prediction == correct_class:
                results.append(True)
            else:
                results.append(False)
                
print("\nPrediction results: \n\n{}".format(results))
accuracy = results.count(True)/len(results)
print("\nAccuracy against model: {:.2%}".format(accuracy))

### Save Model

In [ ]:
from keras.models import model_from_json
from keras.models import load_model

# Serialize model to JSON
model_json = classifier.to_json()


with open("./saved_models/pneumonia_model.json", "w") as json_file:
    json_file.write(model_json)

# Serialize weights to HDF5
classifier.save_weights("./saved_models/pneumonia_model.h5")

### Load Model

In [ ]:
# Load json and create model
json_file = open('./saved_models/pneumonia_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# Load weights into new model
loaded_model.load_weights("./saved_models/pneumonia_model.h5")
print("Loaded model from disk")

### Example usage with loaded model 

In [ ]:
loaded_model.predict(test_image)